## FreeCAD setup

In [1]:
import sys, os

os.environ["QT_API"] = "pyside"

sys.path.append("/usr/lib/freecad-daily-python3/lib/")
sys.path.append("/usr/share/freecad-daily/Mod/Jupyter")

import FreeCAD, FreeCADGui
FreeCADGui.setupWithoutGUI()

## Running a simple test function from the external workbench

In [2]:
import Test
Test.test()

test


## Running a test function that displays an interactive WebGL scene inline (using pythree.js)

In [3]:
Test.display_js_example()

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), quaternion=(0.0, 0.0, 0.0, 1.0), sca…

## Creating an object and outputting it's code to the notebook

### Outputting VRML

In [4]:
from pivy import *
from pivy.gui import soqt
doc = FreeCAD.newDocument()
doc.addObject("Part::Box","Box")
doc.addObject("Part::Cylinder","Cylinder")
doc.addObject("Part::Sphere","Sphere")
doc.addObject("Part::Torus","Torus")
doc.recompute()

from pivy import coin
root = coin.SoSeparator()
for obj in doc.Objects:
    root.addChild(FreeCADGui.subgraphFromObject(obj))

# when exporting to VRML2 the Switches must be replaced
root2 = FreeCADGui.replaceSwitchNodes(root)
tovrml2 = coin.SoToVRML2Action()
tovrml2.apply(root2)
vrmlRoot = tovrml2.getVRML2SceneGraph()

out =  coin.SoOutput()
out.openFile("/tmp/output.vrml")
wa = coin.SoWriteAction(out)
wa.apply(vrmlRoot)
out.closeFile()

In [5]:
help(wa)

Help on SoWriteAction in module pivy.coin object:

class SoWriteAction(SoAction)
 |  Proxy of C++ SoWriteAction class.
 |  
 |  Method resolution order:
 |      SoWriteAction
 |      SoAction
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __del__ lambda self
 |  
 |  __init__(self, *args)
 |      __init__(SoWriteAction self) -> SoWriteAction
 |      __init__(SoWriteAction self, SoOutput out) -> SoWriteAction
 |  
 |  __repr__ = _swig_repr(self)
 |  
 |  __swig_destroy__ = delete_SoWriteAction(...)
 |      delete_SoWriteAction(SoWriteAction self)
 |  
 |  continueToApply(self, *args) -> 'void'
 |      continueToApply(SoWriteAction self, SoNode node)
 |      continueToApply(SoWriteAction self, SoPath path)
 |  
 |  getOutput(self) -> 'SoOutput *'
 |      getOutput(SoWriteAction self) -> SoOutput
 |  
 |  getTypeId(self) -> 'SoType'
 |      getTypeId(SoWriteAction self) -> SoType
 |  
 |  ----------------------------------------------------------------------
 |  Static me

from IPython.display import HTML

input_form = """
<div id="Stats-output">
</div>
<!-- Div which will hold the Output -->
<div id="WebGL-output">
</div>
"""

javascript = """
<script type="text/Javascript">
    // once everything is loaded, we run our Three.js stuff.
    function init() {

        var stats = initStats();


        // create a scene, that will hold all our elements such as objects, cameras and lights.
        var scene = new THREE.Scene();

        // create a camera, which defines where we're looking at.
        var camera = new THREE.PerspectiveCamera(45, window.innerWidth / window.innerHeight, 0.1, 1000);

        // create a render and set the size
        var webGLRenderer = new THREE.WebGLRenderer();
        webGLRenderer.setClearColor(new THREE.Color(0x000, 1.0));
        webGLRenderer.setSize(window.innerWidth, window.innerHeight);
        webGLRenderer.shadowMapEnabled = true;

        // position and point the camera to the center of the scene
        camera.position.x = 30;
        camera.position.y = 30;
        camera.position.z = 30;
        camera.lookAt(new THREE.Vector3(0, 0, 0));

        var orbit = new THREE.OrbitControls(camera);

        var dir1 = new THREE.DirectionalLight(0.4);
        dir1.position.set(-30, 30, -30);
        scene.add(dir1);

        var dir2 = new THREE.DirectionalLight(0.4);
        dir2.position.set(-30, 30, 30);
        scene.add(dir2);

        var dir3 = new THREE.DirectionalLight(0.4);
        dir3.position.set(30, 30, -30);
        scene.add(dir3);

        // add spotlight for the shadows
        var spotLight = new THREE.SpotLight(0xffffff);
        spotLight.position.set(30, 30, 30);
        scene.add(spotLight);

        // add the output of the renderer to the html element
        document.getElementById("WebGL-output").appendChild(webGLRenderer.domElement);

        // call the render function
        var step = 0;


        // setup the control gui
        var controls = new function () {
            // we need the first child, since it's a multimaterial


        };

        var group;
        var gui = new dat.GUI();


        var loader = new THREE.VRMLLoader();
        var group = new THREE.Object3D();
        loader.load("../assets/models/vrml/tree.wrl", function (model) {

            console.log(model);

            model.traverse(function (child) {
                if (child instanceof THREE.Mesh) {
//                    child.material = new THREE.MeshLambertMaterial({color:0xaaaaaa});
                    console.log(child.geometry);
                }
            });

            model.scale.set(10, 10, 10);

            scene.add(model);

        });


        render();


        function render() {
            stats.update();

            orbit.update();

            if (group) {
                group.rotation.y += 0.006;
                // group.rotation.x+=0.006;
            }

            // render using requestAnimationFrame
            requestAnimationFrame(render);
            webGLRenderer.render(scene, camera);
        }

        function initStats() {

            var stats = new Stats();
            stats.setMode(0); // 0: fps, 1: ms

            // Align top-left
            stats.domElement.style.position = 'absolute';
            stats.domElement.style.left = '0px';
            stats.domElement.style.top = '0px';

            document.getElementById("Stats-output").appendChild(stats.domElement);

            return stats;
        }
    }
    window.onload = init;
</script>
"""

HTML(input_form + javascript)


In [6]:
%%HTML



In [9]:
%%HTML
<div id="threejs"/>
<div id="info">
<a href="https://threejs.org" target="_blank" rel="noopener">three.js</a> - VRML loader
</div>

<script type="module">
import * as THREE from 'https://raw.githubusercontent.com/kryptokommunist/Jupyter_FreeCAD/master/Jupyter/js/three.module.js';
import Stats from 'https://raw.githubusercontent.com/kryptokommunist/Jupyter_FreeCAD/master/Jupyter/jsm/libs/stats.module.js';

import { OrbitControls } from 'https://raw.githubusercontent.com/kryptokommunist/Jupyter_FreeCAD/master/Jupyter/jsm/controls/OrbitControls.js';
import { VRMLLoader } from 'https://raw.githubusercontent.com/kryptokommunist/Jupyter_FreeCAD/master/Jupyter/jsm/loaders/VRMLLoader.js';
import { GUI } from 'https://raw.githubusercontent.com/kryptokommunist/Jupyter_FreeCAD/master/Jupyter/jsm/libs/dat.gui.module.js';

var camera, scene, renderer, stats, controls, loader;

var params = {
    asset: 'house'
};

var assets = [
    'creaseAngle',
    'crystal',
    'house',
    'elevationGrid1',
    'elevationGrid2',
    'extrusion1',
    'extrusion2',
    'extrusion3',
    'lines',
    'meshWithLines',
    'meshWithTexture',
    'pixelTexture',
    'points',
];

var vrmlScene;



function init_three() {

    camera = new THREE.PerspectiveCamera( 60, window.innerWidth / window.innerHeight, 0.1, 1e10 );
    camera.position.set( - 10, 5, 10 );

    scene = new THREE.Scene();
    scene.add( camera );

    // light

    var hemiLight = new THREE.HemisphereLight( 0xffffff, 0x000000, 1 );
    scene.add( hemiLight );

    var dirLight = new THREE.DirectionalLight( 0xffffff, 0.5 );
    dirLight.position.set( 200, 200, 200 );
    scene.add( dirLight );

    loader = new VRMLLoader();
    loadAsset( params.asset );

    // renderer

    renderer = new THREE.WebGLRenderer();
    renderer.setPixelRatio( window.devicePixelRatio );
    renderer.setSize( window.innerWidth/2, window.innerHeight/2 );
    var element = document.getElementById('threejs');
    element.appendChild( renderer.domElement );

    // controls

    controls = new OrbitControls( camera, renderer.domElement );
    controls.minDistance = 1;
    controls.maxDistance = 200;
    controls.enableDamping = true;

    //

    stats = new Stats();
    element.appendChild( stats.dom );

    //

    window.addEventListener( 'resize', onWindowResize, false );

    //

    var gui = new GUI( { width: 300 } );
    gui.add( params, 'asset', assets ).onChange( function ( value ) {

        if ( vrmlScene ) {

            vrmlScene.traverse( function ( object ) {

                if ( object.material ) object.material.dispose();
                if ( object.material && object.material.map ) object.material.map.dispose();
                if ( object.geometry ) object.geometry.dispose();

            } );

            scene.remove( vrmlScene );

        }

        loadAsset( value );

    } );

}

function loadAsset( asset ) {

    loader.load( 'https://threejs.org/examples/models/vrml/' + asset + '.wrl', function ( object ) {

        vrmlScene = object;
        scene.add( object );
        controls.reset();

    } );

}

function onWindowResize() {

    camera.aspect = window.innerWidth / window.innerHeight;
    camera.updateProjectionMatrix();

    renderer.setSize( window.innerWidth, window.innerHeight );

}

function animate() {

    requestAnimationFrame( animate );

    controls.update(); // to support damping

    renderer.render( scene, camera );

    stats.update();

}

init_three();
animate();
</script>

In [8]:
%%HTML
<script>alert('Last World!');</script>
<script type="module">;document.getElementById('threejs').innerHTML = "xd"</script>